In [1]:
import os
import sys
import pandas as pd
import requests
import re
from datetime import datetime
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
import time
import json

# 블로그의 정보를 가져오기 위해서 먼저 포스트 제목
# 그리고 포스트 url과 블로그 owner를 데이터 형식으로 가져온다

In [2]:
base_url = 'https://section.blog.naver.com/Search/Post.nhn?pageNo=1&rangeType=ALL&orderBy=sim&keyword=%EA%B0%95%EB%82%A8%20%EB%A7%9B%EC%A7%91'

In [3]:
browser = Chrome()
browser.maximize_window()

In [4]:
browser.get(base_url)

In [6]:
body = browser.find_element_by_tag_name('body')
num_page_down = 2
while num_page_down:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)
    num_page_down -= 1

# 루프문으로 안돌렸을 때

In [63]:
browser.find_elements_by_xpath('//*[@class="pagination"]//a')

[<selenium.webdriver.remote.webelement.WebElement (session="f6ba3aeb3e45d2cafb3bf31e4ff1b7c0", element="0.5372469255151211-57")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f6ba3aeb3e45d2cafb3bf31e4ff1b7c0", element="0.5372469255151211-58")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f6ba3aeb3e45d2cafb3bf31e4ff1b7c0", element="0.5372469255151211-59")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f6ba3aeb3e45d2cafb3bf31e4ff1b7c0", element="0.5372469255151211-60")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f6ba3aeb3e45d2cafb3bf31e4ff1b7c0", element="0.5372469255151211-61")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f6ba3aeb3e45d2cafb3bf31e4ff1b7c0", element="0.5372469255151211-62")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f6ba3aeb3e45d2cafb3bf31e4ff1b7c0", element="0.5372469255151211-63")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f6ba3aeb3e45d2cafb3bf31e4ff1b7

In [114]:
#페이지 자동 넘김
browser.find_elements_by_xpath('//*[@class="pagination"]//a')[1].click()

In [115]:
html_blog0 = browser.page_source

In [116]:
html_blog = BeautifulSoup(html_blog0,'html.parser')

In [117]:
blog_url0 = html_blog.find('div',{'class':'area_list_search'})

In [101]:
blog_url1 = blog_url0.find_all('div',{'class':'list_search_post'})

In [75]:
blog_url = blog_url1[2].find('a')['href']

In [78]:
blog_title0 = blog_url1[1].find('strong',{'class':'title_post'}).find('span')

In [79]:
blog_title0.text

'강남 맛집 매운맛을 원한다면'

# 반복문으로 돌렸을 때

In [147]:
base_url = 'https://section.blog.naver.com/Search/Post.nhn?pageNo=1&rangeType=ALL&orderBy=sim&keyword=%EA%B0%95%EB%82%A8%20%EB%A7%9B%EC%A7%91'
browser = Chrome()
browser.maximize_window()
browser.get(base_url)
blog_data = pd.DataFrame({'blog_title':[],
                         'blog_url':[]})
for i in range(0,20):
    if i<=10:
        #페이지 자동 넘김
        browser.find_elements_by_xpath('//*[@class="pagination"]//a')[i].click()    
        time.sleep(3)#로딩되는 시간까지 기다린 다음 코드를 진행해야 올바른 소스코드를 긁어올 수 있음
        #같은 코드로 browser.implicitly_wait() 가 있음
        
        html_blog0 = browser.page_source
        html_blog = BeautifulSoup(html_blog0,'html.parser')
        blog_url0 = html_blog.find('div',{'class':'area_list_search'})
        blog_url1 = blog_url0.find_all('div',{'class':'list_search_post'})
        
        insert_data = pd.DataFrame({'blog_title':[],
                                     'blog_url':[]})
        for url in range(len(blog_url1)):
            blog_url = blog_url1[url].find('a')['href']
            
            blog_title0 = blog_url1[url].find('strong',{'class':'title_post'}).find('span')
            blog_title = blog_title0.text
            
            insert_data2 = pd.DataFrame({'blog_title':[blog_title],
                                         'blog_url':[blog_url]})
            insert_data = insert_data.append(insert_data2)
    else:
        #페이지 자동 넘김
        browser.find_elements_by_xpath('//*[@class="pagination"]//a')[i-9].click()    
        time.sleep(3)
        
        html_blog0 = browser.page_source
        html_blog = BeautifulSoup(html_blog0,'html.parser')
        blog_url0 = html_blog.find('div',{'class':'area_list_search'})
        blog_url1 = blog_url0.find_all('div',{'class':'list_search_post'})
        
        insert_data = pd.DataFrame({'blog_title':[],
                                     'blog_url':[]})
        for url in range(len(blog_url1)):
            blog_url = blog_url1[url].find('a')['href']
            
            blog_title0 = blog_url1[url].find('strong',{'class':'title_post'}).find('span')
            blog_title = blog_title0.text
            
            insert_data2 = pd.DataFrame({'blog_title':[blog_title],
                                         'blog_url':[blog_url]})
            insert_data = insert_data.append(insert_data2)
            
    blog_data = blog_data.append(insert_data)

In [149]:
blog_data.index = range(len(blog_data))

In [150]:
blog_data

,blog_title,blog_url
0,강남맛집 진구곱창,https://blog.naver.com/ytyt4242/221479981812
1,[강남 회집 맛집] 강진수산물직판장 선릉역,https://blog.naver.com/tobemaven/221484609323
2,[대치동 곰탕 맛집] 하동관 강남분점 #8,https://blog.naver.com/lisso1234/221424687327
3,/맛집이야기-12/ 서울 강남 논현 륭림닭발,https://blog.naver.com/meehwa14/221513448437
4,역삼동맛집 연예인들도 많이 찾는 강남초밥맛집 은행골,https://blog.naver.com/hellorio/221453271907
5,강남 모임장소 팬아시아홈 강남맛집 추천!!!,https://blog.naver.com/hwadlzak/221492789797
6,[역삼역 bills 빌즈 강남] 호주식 브런치 맛집 #1,https://blog.naver.com/lisso1234/221409249998
7,강남 맛집 기분좋은 설레임!,https://blog.naver.com/realwindow2/221487715595
8,강남 맛집 매운맛을 원한다면,https://blog.naver.com/ssong_mom/221508610375
9,강남 맛집 이렇게 다양하게?,https://blog.naver.com/dmsql_0105/221498251145


# 따로따로클릭을 부여해서 소스를 긁어 오는 것은 되는데 반복문으로 돌리게 되면 소스를 긁어오지 못함 - 해결완료로딩페이지를 기다리지 않고 바로 소스코드를 긁어오려고 하니 겉에 묶여있는 소스코드만 긁어와짐

# 각 블로그에 접근하여 본문의 내용 긁어오기

In [151]:
blog_url = blog_data['blog_url'].iloc[0]

In [152]:
browser = Chrome()
browser.maximize_window()

browser.get(blog_url)
browser.implicitly_wait(3)

In [153]:
browser.switch_to_frame(browser.find_element_by_name('mainFrame'))

### 블로그의 내용을 그대로 긁어올 수 있는 방법이 있을까?
### 사진, 글 위치를 그대로 맞춰서
### 현재는 이미지와 텍스트를 따로 긁어와서 정리하는 수밖에 없음

In [154]:
blog_html0 = browser.page_source

In [155]:
blog_html = BeautifulSoup(blog_html0,'html.parser')

In [167]:
blog_text0 = blog_html.find('div',{'id':'postViewArea'})

In [169]:
re.sub('\u200b','',blog_text0.get_text())

'\n\n강남맛집 진구곱창주말에 다녀온 강남~동네에서 버스 타면 환승없이 쭉 가기 때문에편하게 다녀올 수 있었어요.저희는 곱창이 땡겨서 강남 지오다노안쪽으로 들어가서 곱창 맛집에 갔어요.진구곱창 강남역 본점여기\xa0골목길은\xa0지나칠 때마다곱창집 모여있는\xa0걸 본 적 있는데,이렇게 직접 먹어본 건 처음이었어요.가게 입구 쪽에 메뉴판이 있어서미리 메뉴나 가격을 확인할 수 있습니다.바깥에서 본 이미지와 다르게매장 안이 무척 넓더라구요.오후 5시쯤 맞춰서 갔더니아직 저녁시간이 되지 않아바깥쪽에만 손님들이 앉고 안에는 썰렁했었는데요.주문한 이후에 손님들 많이 오더라구요.대기석도 작게 마련되어 있는 걸 보니깐밤에 손님들 몰릴 때는 꽉 차서 대기해야할 수도 있나 봅니다.오뎅탕과 밑반찬~1++ 한우모듬 중 사이즈를 먼저 주문하고,콜라, 맥주도 하나씩~요즘 술을 자제하고 있지만,소곱창 먹을 때 맥주 한잔도안하는 건 너무 아쉽죠..!강남맛집 한우모듬~소곱창은 직원 분이 직접 구워 주면서알맞게\xa0잘라주십니다.염통 먼저 빠르게 먹어주고,곱창과 막창, 대창은 완전히 익을 때까지좀더 기다려 줬습니다.양은 생각보다 많지 않았지만,강남맛집 위치를 생각하면 이해는 되더라구요.양파절임이나 쏭쏭 썰은 고추가 들어간 소스에곁들여 먹으면 곱창을 더 맛있게 먹을 수 있어요.중간에 소스랑 양파절임 좀 더 달라고 말씀드렸더니아예 다른 그릇에 가득 채워서 넉넉히 주시더라구요.세 명이 갔는데 다 배가 출출할 때라서순식간에 해치웠어요..곱창 먹어줬으니 다음으로 막창 주문했습니다.막창은 소금, 양념 반반도 주문된다는데,저희는 1인분씩 주문했어요!막창도 양이 조금 적긴 했지만,가격 자체가 9,000원으로 저렴한 편!2인분이 다른 곳 1.5인분 가격 정도이니괜찮은 것 같기도..?!진구곱창은 소곱창 말고 막창도 괜찮더라구요.막창을 워낙 좋아하는 저라서맛있게 잘 먹었습니다.강남맛집 마지막 마무리는 볶음밥!!!여기는 일반 볶음밥과 다르게깍두기가 들어간 볶음밥이에요.인스타 부메랑으로 찍은 영상직원 분이 눈앞